In [4]:
# Importing and starting a spark session
from pyspark.sql import SparkSession
from pyspark import SparkContext
#Supress warnings
spark = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate()
sc.setLogLevel('WARN')

spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)
#Make the spark files present well
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

21/08/14 16:33:24 WARN Utils: Your hostname, LAPTOP-D5HGLKLK resolves to a loopback address: 127.0.1.1; using 172.18.37.131 instead (on interface eth0)
21/08/14 16:33:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/14 16:33:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
taxi_2018 = spark.read.parquet('../preprocessed_data/taxi_2018.parquet')
taxi_2019 = spark.read.parquet('../preprocessed_data/taxi_2019.parquet')

In [6]:
from pyspark.sql.functions import *
taxi_2018.groupBy("payment_type").agg(mean('tip_amount'))

payment_type,avg(tip_amount)
1,2.7005132659386417
3,1.488654511390633...
5,4.016666666666667
4,-0.00600901659131...
2,6.643742587460488E-5


We are only interested in the amount of tip received by the driver for any trip. As tips can only be received via credit card (id = 1), all other ids are not of interest to us and thus are removed.

In [7]:
taxi_2018.filter(taxi_2018.passenger_count == 0).limit(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
1,2018-04-01 00:20:09,2018-04-01 00:24:44,0,0.8,1,N,50,48,1,5.0,0.5,0.5,1.0,0.0,0.3,7.3
1,2018-04-01 00:43:37,2018-04-01 00:50:21,0,0.9,1,N,113,79,1,6.0,0.5,0.5,1.46,0.0,0.3,8.76
1,2018-04-01 00:46:34,2018-04-01 00:52:51,0,0.9,1,N,144,114,1,6.0,0.5,0.5,0.0,0.0,0.3,7.3
1,2018-04-01 00:05:26,2018-04-01 00:43:16,0,18.0,2,N,132,143,1,52.0,0.0,0.5,9.2,0.0,0.3,62.0
1,2018-04-01 00:54:24,2018-04-01 01:01:03,0,1.4,1,N,163,164,2,7.0,0.5,0.5,0.0,0.0,0.3,8.3


In [8]:
taxi_2018.groupBy('passenger_count').count()

passenger_count,count
192,1
1,73072141
6,2783068
3,4295075
96,1
5,4602861
9,275
4,2029082
8,313
7,390


On the other hand, we will remove trips > 10 passengers, as they are likely errornerous and may have a negative effect on skewing our data.

In [9]:
taxi_2018.groupBy('RatecodeID').count()

RatecodeID,count
1,99727994
6,868
3,209560
5,413027
4,64830
2,2382882
99,5089


Given its small scale, we can remove the trips with RatecodeID of 99 as they do not meet data integrity requirements. We will also remove all taxi entries with RatecodeID 3 (Newark Airport) and Ratecode 4 (Nassau/Westchester), as these do not have the taxi zone data available, and are also charged a different way (JFK trips are also charged differently, but occur frequently and is still contained in New York City.) 

https://www1.nyc.gov/site/tlc/passengers/taxi-fare.page

Next, we make sure the LocationIDs are valid (the NYC Taxi Zone lookup has values 1->263)

In [10]:
LocID_analysis = taxi_2018.groupBy('PULocationID').count().toPandas()

In [11]:
#creating an index of Location IDs from the taxi zone lookup table to find conflicting values
location_index = [row['OBJECTID'] for index, row in taxi_zones.iterrows()]
location_test = []
for index, row in LocID_analysis.iterrows():
    if row['PULocationID'] not in location_index: 
        location_test.append(row['PULocationID'])
    else:
        location_index.remove(row['PULocationID'])
print(location_test)

[264, 265]


In [12]:
print(LocID_analysis.loc[LocID_analysis['PULocationID'] == 264])
print(LocID_analysis.loc[LocID_analysis['PULocationID'] == 265])

     PULocationID    count
154           264  1613394
     PULocationID  count
196           265  49636


The 264 and 265 LocationIDs are in the lookup table under borough Unknown but not in the shapefile of the zones.
This, along with their large count (especially in ID 264), suggests that we are looking at the trips which are outside 
of the city of New York. We will remove this data, as it not only doesn't strictly follow data dictionary guidelines, but also is outside the the scope of the investigation (New York City)

All non-2018 years will also be removed without further investigation (observed from manual viewing of data)

Negative amounts in any charge will be removed as well.

We will also remove all columns relating to fare that is not tip amount - things like fare amount or total amount are already correlated by definition and are not interesting to us. The payment type will also be removed as we have already filtered it out.